The way this works is I convert the merged and filtered founder vcf to a hapsample fileset and then I replace the genotypes in that fileset with the actual read counts for each allele, hence generating the AHMM ancestral haplotypes.

I need to watch out for unexpected symbols in the mpileup. I noticed that in the TRD data and have a replacement script ready to get rid of that stuff if need be.

In [1]:
# set important data in the config file first!
source("config.R")
source("../../extra-R-functions.R")
library(naturalsort)

v Reading from Status of mouse sequencing.

v Range ''All founder files''.

v Reading from Status of mouse sequencing.

v Range ''All Fx files''.


Attaching package: 'tidytable'


The following objects are masked from 'package:stats':

    dt, filter, lag


The following object is masked from 'package:base':

    %in%




In [2]:
# first we create the hapsample file, which is a format very close to AncestryHMM input
if(!file.exists(paste0(folder_for_sequences,"Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.hap.gz"))){
    cmd=paste0("sh -xe ",Barn_Mice_dir,"01_Genotyping/01_Founders/06_MakeHapsFile.sh ", folder_for_sequences, " ", "Founders.filtered.var")

    execute_cmd_sbatch(cmd, jobname="HapSample", env=env_mapping_etc,cpu="1", activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"),
                        time="short", mem="16G", acc=slurm_acc)
    Sys.sleep(5)

    while(slurm_check_jobs_still_running(columbia_username,"HapSample")){
        Sys.sleep(60)
    }
}


[1] "sbatch -c 1 --mem=16G --job-name=HapSample -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; sh -xe ~/ColumbiaProjects/Barn_Mice/01_Genotyping/01_Founders/06_MakeHapsFile.sh /moto/ziab/users/jr3950/data/genomes/tmp_founders/ Founders.filtered.var'"
[1] "Submitted batch job 13584227"


In [3]:
# get the mpileup per founder for the loci of interest
files=paste0(bam_dir,"/","SW_",1:12,".merged.bam")
if(sum(!file.exists(files))>0){
    stop("Missing BAM files!")
}
for(bam in files){
    sample=strsplit(basename(bam),".",fixed=TRUE)[[1]][1]
    if(file.exists(paste0(folder_for_sequences,sample,".mpileup.xz"))){
        #next
    }
    cmd=paste0("samtools mpileup -l ",folder_for_sequences,"Founders.filtered.posfile_for_mpileup ",
           "-f ",ref_file," ",bam," | xz > ",folder_for_sequences,sample,".mpileup.xz")
    execute_cmd_sbatch(cmd, jobname="Mpileup", env=env_mapping_etc,cpu="1", activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"),
                        time="short", mem="8G", acc=slurm_acc)
    Sys.sleep(1)
}
while(slurm_check_jobs_still_running(columbia_username,"Mpileup")){
        Sys.sleep(60)
    }

[1] "sbatch -c 1 --mem=8G --job-name=Mpileup -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; samtools mpileup -l /moto/ziab/users/jr3950/data/genomes/tmp_founders/Founders.filtered.posfile_for_mpileup -f /moto/ziab/users/jr3950/data/genomes/GCF_000001635.26_GRCm38.p6_genomic.fna /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_1.merged.bam | xz > /moto/ziab/users/jr3950/data/genomes/tmp_founders/SW_1.mpileup.xz'"
[1] "Submitted batch job 13584228"
[1] "sbatch -c 1 --mem=8G --job-name=Mpileup -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; samtools mpileup -l /moto/ziab/users/jr3950/data/genomes/tmp_founders/Founders.filtered.posfile_for_mpileup -f /moto/ziab/users/jr3950/data/genomes/GCF_000001635.26_GRCm38.p6_genomic.fna /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_2.merged.bam | xz > /moto/ziab/users/jr3950/data/genomes/tmp_founders/SW_2.mpileup.xz'"
[1] "Submitted batch job 13584229"


In [6]:
# below I am merging in a bit of a complicaed manner the hapsample file with the mpileup data, so that I have columns of haplotypes with
# the respective counts of alleles to feed into AHMM as the ancestries

mpileups=naturalsort(list.files(path=folder_for_sequences, pattern="mpileup.xz", full.names=TRUE))

writeLines(mpileups, mpileups_path<-paste0(folder_for_sequences,"mpileups.list"))

cmd=paste0("Rscript ",Barn_Mice_dir,"01_Genotyping/01_Founders/06_MakeAncestries.R ", mpileups_path, " ",
           paste0(folder_for_sequences,"Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.hap.gz"), " ",
           paste0(folder_for_sequences,"Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.samples"), " ",
           paste0(folder_for_sequences,"Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.hap.prepped.gz"), " ",
           "10")


execute_cmd_sbatch(cmd, jobname="MakeAncestries", env=env_jupyter,cpu="10", activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"),
                        time="long", mem="40G", acc=slurm_acc)

[1] "sbatch -c 10 --mem=40G --job-name=MakeAncestries -A ziab -t 119:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh my_conda4; Rscript ~/ColumbiaProjects/Barn_Mice/01_Genotyping/01_Founders/06_MakeAncestries.R /moto/ziab/users/jr3950/data/genomes/tmp_founders/mpileups.list /moto/ziab/users/jr3950/data/genomes/tmp_founders/Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.hap.gz /moto/ziab/users/jr3950/data/genomes/tmp_founders/Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.samples /moto/ziab/users/jr3950/data/genomes/tmp_founders/Founders.filtered.var.AllPhasedOrUnphasedHet_hapsample.hap.prepped.gz 10'"
[1] "Submitted batch job 13584277"


In [5]:
while(slurm_check_jobs_still_running(columbia_username,"MakeAncestries")){
        Sys.sleep(60)
    }

# now we have the fouder input data for AHMM. ignore the cM, it will be added later!

head(fread(paste0(folder_for_sequences,"Founders.filtered.AllPhasedOrUnphasedHet_hapsample.hap.prepped.gz")))

chr,bp,ref,alt,1_1a,1_1b,1_2a,1_2b,2_1a,2_1b,...,10_2b,11_1a,11_1b,11_2a,11_2b,12_1a,12_1b,12_2a,12_2b,cM
<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,3003414,A,G,0,26,0,26,0,19,...,19,0,17,0,17,0,10,0,10,0
1,3008615,A,G,0,10,0,10,0,8,...,8,0,7,0,7,0,10,0,10,0
1,3008701,T,C,0,13,0,13,0,17,...,10,0,14,0,14,0,8,0,8,0
1,3008712,G,A,11,0,11,0,19,0,...,0,18,0,18,0,9,0,9,0,0
1,3008739,T,C,0,10,0,10,0,16,...,8,0,16,0,16,0,8,0,8,0
1,3011102,C,T,0,18,0,18,0,11,...,8,0,17,0,17,0,10,0,10,0


In [6]:
# remove temporary files

# remove mpileups, as new concise ones will be created for the loci of interest
files_to_remove=list.files(path=bam_dir, pattern=".mpileup$", full.names = TRUE)
# Move the unphased chrY X MT merged as FoundersAndF1.unphased.unfilterd.XYMT.bcf to the main dir
files=paste0(bam_dir,"/Founders.chr",c("X","Y","MT"),".vcf.gz")
if(sum(!file.exists(files))>0){
    stop("missing files")
}
samples=list()
for(f in files){
    samples[[f]]=readLines(paste0(f,".samples"))
    print(samples[[f]])
}

are_all_vectors_identical <- all(sapply(samples[-1], identical, samples[[1]]))

print(are_all_vectors_identical)

if(!are_all_vectors_identical){
    stop("Assumption of same samples / order in each vcf not met. Please fix.")
}

cmd=paste0("bcftools concat -Ob -o ",folder_for_sequences,"FoundersAndF1.unphased.unfiltered.XYMT.bcf --threads 4 ",paste(files,collapse=" "))
execute_cmd_sbatch(cmd, mem="8gb", cpu="4", time="short", env=env_mapping_etc, jobname="MergeVCF3",
                              activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh")) 
    Sys.sleep(5)
    

while(slurm_check_jobs_still_running(columbia_username,"MergeVCF3")){
        Sys.sleep(60)
    }


# remove all vcf and their indices from the bam/ subdir as we have the merged ones now
files_to_remove=c(files_to_remove,list.files(path=bam_dir, pattern=".vcf.gz$", full.names = TRUE))
files_to_remove=c(files_to_remove,list.files(path=bam_dir, pattern=".vcf.gz.csi$", full.names = TRUE))

files_to_remove

file.remove(files_to_remove)

 [1] "SM_SW_8"    "SM_SW_7"    "SM_SW_11"   "SM_SW_10"   "SM_SW_4"   
 [6] "SM_SW_1"    "SM_SW_2"    "SM_SW_5"    "SM_SW_25"   "SM_SW_9"   
[11] "SM_SW_12"   "SM_SW_6"    "SM_SW_3"    "SM_SW_1133" "SM_SW_31"  
[16] "SM_SW_80"   "SM_SW_85"   "SM_SW_87"   "SM_SW_90"   "SM_SW_84"  
 [1] "SM_SW_8"    "SM_SW_7"    "SM_SW_11"   "SM_SW_10"   "SM_SW_4"   
 [6] "SM_SW_1"    "SM_SW_2"    "SM_SW_5"    "SM_SW_25"   "SM_SW_9"   
[11] "SM_SW_12"   "SM_SW_6"    "SM_SW_3"    "SM_SW_1133" "SM_SW_31"  
[16] "SM_SW_80"   "SM_SW_85"   "SM_SW_87"   "SM_SW_90"   "SM_SW_84"  
 [1] "SM_SW_8"    "SM_SW_7"    "SM_SW_11"   "SM_SW_10"   "SM_SW_4"   
 [6] "SM_SW_1"    "SM_SW_2"    "SM_SW_5"    "SM_SW_25"   "SM_SW_9"   
[11] "SM_SW_12"   "SM_SW_6"    "SM_SW_3"    "SM_SW_1133" "SM_SW_31"  
[16] "SM_SW_80"   "SM_SW_85"   "SM_SW_87"   "SM_SW_90"   "SM_SW_84"  
[1] TRUE
[1] "sbatch -c 4 --mem=8gb --job-name=MergeVCF3 -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; bcftools concat

[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.mpileup"                                                                                                                               
  [2] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr10.mpileup"                                                                                                                              
  [3] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr11.mpileup"                                                                                                                              
  [4] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr12.mpileup"                                                                                                                              
  [5] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr13.mpileup"                                                                                                                              
  [6] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr14.mpileup"                                                                                                                              
  [7] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr15.mpileup"                                                                                                                              
  [8] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr16.mpileup"                                                                                                                              
  [9] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr17.mpileup"                                                                                                                              
 [10] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr18.mpileup"                                                                                                                              
 [11] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr19.mpileup"                                                                                                                              
 [12] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr2.mpileup"                                                                                                                               
 [13] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr3.mpileup"                                                                                                                               
 [14] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr4.mpileup"                                                                                                                               
 [15] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr5.mpileup"                                                                                                                               
 [16] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr6.mpileup"                                                                                                                               
 [17] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr7.mpileup"                                                                                                                               
 [18] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr8.mpileup"                                                                                                                               
 [19] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr9.mpileup"                                                                                                                         

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[136] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[151] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[166] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[181] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[196] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[211] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[226] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[241] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[256] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[271] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[286] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[301] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[316] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[331] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[346] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[361] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [7]:
stop("Not removed bam files here yet. Do I still need them? Put to Engram temporarily (just me while creating this notebook)")

ERROR: Error in eval(expr, envir, enclos): Not removed bam files here yet. Do I still need them? Put to Engram temporarily (just me while creating this notebook)
